In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt  #이미지 확인용 모듈
import matplotlib.image as mpimg  #이미지 확인용 모듈
import os
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten # layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator  #데이터 확장 모듈
from tensorflow import data as tf_data
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential

import glob

import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path = r"/content/drive/MyDrive/IMS 인공지능 스터디/Dataset"

In [3]:
image_paths = sorted([
    os.path.join(path, fname)
    for fname in os.listdir(path)
    if (fname.lower().endswith('.jpg') or fname.lower().endswith('.png'))
      and ('o' in fname.lower() or 'x' in fname.lower())
])

# 이미지 처리 함수
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=1)   # 자동 형식 감지 (JPEG/PNG)
    image.set_shape([None, None, 1])                   # shape 명시

    image = tf.image.resize(image, [28, 28])         # 크기 조절
    image /= 255.0                                   # 정규화
    return image

# tf.data.Dataset 생성
path_ds = tf.data.Dataset.from_tensor_slices(image_paths) #['dataset/o1.png','dataset/o2.png', ...]
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# 라벨 생성 (o -> 0, x -> 1)
labels = [0 if 'o' in os.path.basename(p).lower() else 1 for p in image_paths]

#추가
images = np.array(list(image_ds.as_numpy_iterator()))

#[
#   image1: (28, 28, 1),
#   image2: (28, 28, 1),
 # ...
#]
labels = np.array(labels)

# 데이터 셔플
idx = np.random.permutation(len(images))
images = images[idx]
labels = labels[idx]

# 분할 지점 계산
total = len(images)
train_end = int(0.8 * total)
val_end = int(0.9 * total)

# 분할
train_images, val_images, test_images = images[:train_end], images[train_end:val_end], images[val_end:]
train_labels, val_labels, test_labels = labels[:train_end], labels[train_end:val_end], labels[val_end:]


# npz로 저장
np.savez_compressed('datasets_split.npz',
                    train_images=train_images, train_labels=train_labels,
                    val_images=val_images, val_labels=val_labels,
                    test_images=test_images, test_labels=test_labels)

In [4]:
# npz 로드
data = np.load('datasets_split.npz')
train_images = data['train_images']
train_labels = data['train_labels']
val_images = data['val_images']
val_labels = data['val_labels']
test_images = data['test_images']
test_labels = data['test_labels']

# TensorFlow Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(1000).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

In [5]:
# 클래스 이름 정의
class_names = ['o', 'x']  # 클래스 a와 b

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),  # 입력 이미지를 1차원 배열로 변환
    tf.keras.layers.Dense(4, activation='relu'),    # 4개의 유닛을 가진 Dense 층
    tf.keras.layers.Dense(2)                        # 최종 출력을 위한 2개의 유닛을 가진 Dense 층
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# 모델 컴파일
model.compile(optimizer='adam',  # 옵티마이저로 'adam' 사용
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # 손실 함수
              metrics=['accuracy'])  # 평가 지표로 'accuracy' 사용

# 모델 학습
model.fit(train_dataset, epochs=7, validation_data=val_dataset)  # 모델을 훈련 데이터셋으로 3번 에포크 동안 학습

# 모델 평가
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)  # 테스트 데이터셋으로 모델 평가
print(f"\nTest accuracy: {test_acc:.4f}")  # 테스트 정확도 출력

# 테스트셋에서 하나의 이미지와 라벨 가져오기
for test_image, true_label in test_dataset.take(1):
    test_image = test_image[0]        # (28, 28, 1)
    true_label = true_label[0]        # 정수형 라벨

# 배치 차원 추가 (모델 입력 형태 맞추기)
image_batch = tf.expand_dims(test_image, 0)  # shape: (1, 28, 28)

# 모델 예측 수행
predictions = model.predict(image_batch)

# 예측 결과 해석
predicted_label = class_names[np.argmax(predictions[0])]
actual_label = class_names[true_label.numpy()]

print(f"예측 결과: '{predicted_label}', 실제 라벨: '{actual_label}'")


Epoch 1/7
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.5529 - loss: 0.6878 - val_accuracy: 0.3958 - val_loss: 0.7039
Epoch 2/7
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5230 - loss: 0.6866 - val_accuracy: 0.3958 - val_loss: 0.7033
Epoch 3/7
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5443 - loss: 0.6835 - val_accuracy: 0.3958 - val_loss: 0.7044
Epoch 4/7
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5448 - loss: 0.6806 - val_accuracy: 0.3958 - val_loss: 0.7006
Epoch 5/7
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6256 - loss: 0.6776 - val_accuracy: 0.4167 - val_loss: 0.6980
Epoch 6/7
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5779 - loss: 0.6757 - val_accuracy: 0.3958 - val_loss: 0.6998
Epoch 7/7
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5830 - loss: 0.6734 - val_accuracy: 0.3958 - val_loss: 0.7072
2/2 - 0s - 7ms/step - accuracy: 0.5625 - loss: 0.6757

Test accuracy: 0.5625
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
예측 결과: '

In [7]:
model.save('iris.h5')